In [1]:
import pandas as pd
import os
import sys
import shutil
import dask.dataframe as dd
from tqdm import tqdm


In [2]:
db_dir = "/media/dan/Data/databases"
parquet_path = os.path.join(db_dir, "prediction_table.parquet")

# create a table for the patients
if not os.path.exists(os.path.join(db_dir, "prediction_table.parquet")):
    df =pd.read_csv(os.path.join(db_dir, "predict_df_4NetMets_20250319.csv"))
    df.to_parquet(parquet_path, compression='snappy')
    df = dd.read_parquet(parquet_path)
else:
    df = dd.read_parquet(parquet_path)

In [ ]:
csvs = os.listdir(db_dir)
csvs = [x for x in csvs if "NETWORKSTATS" in x]

held_cols = ['x', 'y', 'z', 'soz', 'pid', 'time', 'electrode_idx']
others = [x for x in df.columns.values if x not in held_cols]
metrics = list(set([x.split("~")[0] for x in others]))

csvs = [x for x in csvs if x.split("~")[1].split(".")[0] not in metrics]

for f in tqdm(csvs):
    tmp = dd.read_csv(os.path.join(db_dir, f))
    merged = df.merge(tmp, on=held_cols, how="outer")
    tmp_path = os.path.join(db_dir, "temp.parquet")
    merged.to_parquet(tmp_path, compression='snappy')
    # check if parquet_path is a file or a directory
    if os.path.isfile(parquet_path):
        os.remove(parquet_path)
    elif os.path.isdir(parquet_path):
        shutil.rmtree(parquet_path)
    os.rename(tmp_path, parquet_path)
    del merged, tmp
    df = dd.read_parquet(parquet_path)


 79%|███████▉  | 15/19 [1:09:00<21:29, 322.28s/it]

In [ ]:
import pyarrow.parquet as pq

# Replace 'your_file.parquet' with the actual path to your Parquet file
parquet_file = pq.ParquetFile(path)

# Get the schema of the Parquet file
schema = parquet_file.schema

# Extract column names from the schema
column_names = schema.names

# Print the list of column names
print(column_names)

In [12]:
# read only soz pid and x columns
p = pq.read_table(path, columns=['soz', 'pid', 'x'])


In [ ]:
import numpy as np
test = np.arange(len(df))


# I want to simulate adding a new column to the parquet file. pretend the parquet file
# is too large to read into memory.
